# Coursera Capstone: Segmenting and Clustering Neighborhoods in Toronto

## Introduction:

In this assigment we will donwload the information of the neighborhoods in Toronto from the Wikipedia and explore it.

# PART 1

## Setting the librarires that we will need for this analysis

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## **Getting data from web page**

We will read the page and convert the table into a dataframe using the pandas read_html method.

In [2]:
#get Toronto neighborhoods from Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#read the Wikipedia page - returns list of dataframes
dfs_wiki = pd.read_html(url, header=0)

#take the first dataframe from the returned list (it should be the only dataframe in the list)
df = dfs_wiki[0]
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#### Now we delete all rows that have "Not Assigned" in Borough values

In [3]:
#create new dataframe with records where Borough is not 'Not assigned'
df_assigned = df[df['Borough'] != 'Not assigned']
df_assigned.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### In rows where the value of Neighbourhood is 'Not assigned' we will replace it with the value of the Borough.

In [4]:
#create a list of neighborhoods, replacing the borough where neighborhood is 'Not assigned'
new_neigh = df_assigned['Neighbourhood'].where(df_assigned['Neighbourhood'] != 'Not assigned', other = df_assigned['Borough'], axis = 0)
#construct new dataframe using postcode and borough from the previous dataframe and neighborhood from the above list
df_replaced = pd.concat([df_assigned['Postal Code'], df_assigned['Borough'], new_neigh], axis = 1)
df_replaced.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Whenever we have more than one row per postcode, we will concatenate all neighborhoods into a comma separated list

In [5]:
#group the dataframe by Postcode and Borough and concatenate all neighborhoods into comma separated list
toronto_neighborhoods = df_replaced.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x: ', '.join(x)).to_frame()
toronto_neighborhoods.reset_index(inplace = True)
toronto_neighborhoods.head(10)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
#display the shape of the resulting dataframe
toronto_neighborhoods.shape

(103, 3)

# PART 2

### ** Geographical location of Neighborhoods **

In this part we are getting the latitude and the longitude coordinates of each neighborhood and add that info to the dataframe we are working on 

First we get the info from the latitudes and longitudes from the csv files

In [7]:
#read provided dataset
coords = pd.read_csv('http://cocl.us/Geospatial_data')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In this step we combine the previous info to the dataframe using merge function

In [8]:
toronto_neighborhoods = pd.merge(toronto_neighborhoods, coords, left_on = 'Postal Code', right_on = 'Postal Code')[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
toronto_neighborhoods.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [9]:
toronto_neighborhoods.shape

(103, 5)

# PART 3 - Analysis of neighborhoods in Toronto

Now I am goiny to use the Foursquare API to explore the neighborhoods of Toronto like in New York's task

** First I define API credentials **

In [10]:
## Code deleted 

** Explore the first neighborhood in the dataframe **

In [11]:
neighborhood_name = toronto_neighborhoods.loc[0, 'Neighbourhood'] # neighborhood name
neighborhood_latitude = toronto_neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


** Now we get the top 100 venues that are in the above neighborhood within a radious of 500 meters **

In [18]:
CLIENT_ID = 'xx' # your Foursquare ID
CLIENT_SECRET = 'xx' # your Foursquare Secret
VERSION = '20201010' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xx
CLIENT_SECRET:xx


In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [14]:
results = requests.get(url).json()

Before going on we will create the get_category_type function which extracts the category name from a JSON object

In [15]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
#extract the items key from the results
venues = results['response']['groups'][0]['items']
#flatten JSON into a dataframe
nearby_venues = json_normalize(venues) 
#filter columns that we need for further analysis
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
#extract the category for each row using the previously defined function
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns ?????????????????????????????????????????????
#nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

KeyError: 'groups'

Create a function to repeat the same process as above to all the neighborhoods in Toronto¶

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The following code executes the above function on each neighborhood and creates a new dataframe called toronto_venues.

In [28]:
toronto_venues = getNearbyVenues(names = toronto_neighborhoods['Neighbourhood'],
                                   latitudes = toronto_neighborhoods['Latitude'],
                                   longitudes = toronto_neighborhoods['Longitude']
                                  )

Malvern, Rouge


KeyError: 'groups'

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(2126, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Count number of venue categories by neighborhood

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,4,4,4,4,4,4
Woodbine Heights,6,6,6,6,6,6


In [21]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 unique categories.


We see that many neighborhoods have fewer than 10 different venue categories. It doesn't make sense to include these in segmentation because there is not enough data to characterize them. Therefore we will exclude these from the dataset.

But first, we have to check how many neighborhoods have fewer than 10 venue categories to ensure that we still have enough neighborhoods left for segmentation.

In [22]:
#store the results of the above counts into a dataframe
toronto_venues_count = toronto_venues.groupby('Neighborhood').count()
print('There are {} neighborhoods with fewer than 10 venue categories.'.format(toronto_venues_count[toronto_venues_count['Venue Category'] < 10].shape[0]))

There are 47 neighborhoods with fewer than 10 venue categories.


Therefore we will exclude the neighborhoods with fewer than 10 venue categories.

In [23]:

#create list with neighborhoods to exclude
neigh_to_exclude = toronto_venues_count[toronto_venues_count['Venue Category'] < 10].index.tolist()
#create filtered dataframe by excluding neighborhoods in above list
toronto_venues_filt = toronto_venues[~toronto_venues['Neighborhood'].isin(neigh_to_exclude)]
#check size of resulting dataframe
toronto_venues_filt.groupby('Neighborhood').count().shape

(49, 6)

The number of neighborhoods is sufficient for further analysis. We will rename the filtered dataset toronto_venues_filt back to the original dataset toronto_venues.

In [24]:
toronto_venues = toronto_venues_filt

## Analysis of each Neighorhood

We will do one hot encoding to pivot category values into columns of the dataframe.

There is one observation that we have to be careful about: one of the category values is Neighborhood. After one hot encoding, this value will become a column name. We are already using the column Neighborhood to represent the neighborhood name. To avoid confusing these columns, we will rename the column that comes from one hot encoding as Neighborhood Category.

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#rename the column 'Neighborhood' which represents a category name to 'Neighborhood Category' 
#this is to distinguish this column from the 'Neighborhood' column which we want to continue to use as the neighborhood name
toronto_onehot.rename(columns={'Neighborhood':'Neighborhood Category'}, inplace=True)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()

In [26]:
toronto_onehot.shape

(1934, 251)

** We will group rows by neighborhood and by taking the mean of the frequency of occurrence of each category **

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1,"Bedford Park, Lawrence Manor East",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.041667,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
2,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.017241,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
3,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
4,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.058824
5,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.0625,0.125,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
6,Canada Post Gateway Processing Centre,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.076923,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
7,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.016949,0.0,0.0,0.000000,0.0,0.016949,0.0,0.0,0.016949
8,Christie,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
9,Church and Wellesley,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012821,0.0,...,0.0,0.000000,0.0,0.0,0.012821,0.0,0.000000,0.0,0.0,0.025641


In [28]:
toronto_grouped.shape

(49, 251)

** Store the above into a pandas dataframe **

Write a function to sort the venues in descending order

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Pharmacy,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Restaurant,Diner,Sandwich Place
1,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Comfort Food Restaurant,Thai Restaurant,Boutique,Juice Bar,Restaurant,Butcher,Pub
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Bakery,Cheese Shop,Restaurant,Eastern European Restaurant,Beach
3,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
4,"Business reply mail Processing Centre, South C...",Yoga Studio,Light Rail Station,Butcher,Burrito Place,Skate Park,Fast Food Restaurant,Auto Workshop,Farmers Market,Garden Center,Garden


In [31]:
neighborhoods_venues_sorted.groupby(['1st Most Common Venue']).size()

1st Most Common Venue
Airport Lounge           1
Bakery                   1
Bar                      1
Breakfast Spot           1
Café                     5
Clothing Store           2
Coffee Shop             20
Fast Food Restaurant     2
Greek Restaurant         1
Grocery Store            4
Gym                      2
Indian Restaurant        1
Mexican Restaurant       1
Park                     1
Pizza Place              1
Ramen Restaurant         1
Sandwich Place           2
Sporting Goods Shop      1
Yoga Studio              1
dtype: int64

## Clustering the neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [32]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=3).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 2, 2, 2, 0, 2, 1, 1, 3, 2, 0, 2, 0, 0, 2, 3, 0, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 1, 2, 2, 0, 2, 2, 0, 2, 2, 2, 1,
       2, 0, 2, 2, 2])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods_venues_sorted

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_neighborhoods.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
0,0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Pharmacy,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Restaurant,Diner,Sandwich Place,M3H,North York,43.754328,-79.442259
1,2,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Comfort Food Restaurant,Thai Restaurant,Boutique,Juice Bar,Restaurant,Butcher,Pub,M5M,North York,43.733283,-79.419750
2,2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Bakery,Cheese Shop,Restaurant,Eastern European Restaurant,Beach,M5E,Downtown Toronto,43.644771,-79.373306
3,2,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,M6K,West Toronto,43.636847,-79.428191
4,0,"Business reply mail Processing Centre, South C...",Yoga Studio,Light Rail Station,Butcher,Burrito Place,Skate Park,Fast Food Restaurant,Auto Workshop,Farmers Market,Garden Center,Garden,M7Y,East Toronto,43.662744,-79.321558


In [34]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Note: because GitHub doesn't display Folium maps, a print screen of the map is available [here](https://github.com/CasteSan/Coursera_Capstone/blob/main/map_data_W3.png)

## Clusters analysis

We will examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

### Cluster 0: Multicultural zone

In [36]:
toronto_cluster0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]-2))]]
toronto_cluster0

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Pharmacy,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Restaurant,Diner,Sandwich Place,M3H,North York
4,"Business reply mail Processing Centre, South C...",Yoga Studio,Light Rail Station,Butcher,Burrito Place,Skate Park,Fast Food Restaurant,Auto Workshop,Farmers Market,Garden Center,Garden,M7Y,East Toronto
10,"Clarks Corners, Tam O'Shanter, Sullivan",Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Italian Restaurant,Thai Restaurant,Convenience Store,Chinese Restaurant,Bank,Gas Station,Pharmacy,M1T,Scarborough
12,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Café,Gym,Sushi Restaurant,Deli / Bodega,Indian Restaurant,M4S,Central Toronto
13,Davisville North,Gym,Hotel,Pizza Place,Dance Studio,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gym / Fitness Center,M4P,Central Toronto
16,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Bank,Brewery,Park,Grocery Store,Café,Music Venue,Supermarket,Bar,M6H,West Toronto
22,"India Bazaar, The Beaches West",Park,Pizza Place,Sandwich Place,Fast Food Restaurant,Ice Cream Shop,Pet Store,Movie Theater,Pub,Restaurant,Burrito Place,M4L,East Toronto
27,"Mimico NW, The Queensway West, South of Bloor,...",Grocery Store,Tanning Salon,Convenience Store,Discount Store,Burrito Place,Burger Joint,Sandwich Place,Kids Store,Supplement Shop,Bakery,M8Z,Etobicoke
31,"Parkview Hill, Woodbine Gardens",Pizza Place,Gym / Fitness Center,Flea Market,Bank,Athletics & Sports,Gastropub,Intersection,Café,Pharmacy,Breakfast Spot,M4B,East York
36,"South Steeles, Silverstone, Humbergate, Jamest...",Grocery Store,Pizza Place,Video Store,Sandwich Place,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Pharmacy,Donut Shop,Dumpling Restaurant,M9V,Etobicoke


This could be a multicultural neighborhood with restaurants of cuisines of all around the globe

### Cluster 1: Expensive neighbour

In [37]:
toronto_cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]-2))]]
toronto_cluster1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough
6,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Gym,American Restaurant,Intersection,Middle Eastern Restaurant,Burrito Place,Sandwich Place,Fried Chicken Joint,Mediterranean Restaurant,M7R,Mississauga
7,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Middle Eastern Restaurant,Portuguese Restaurant,Poke Place,M5G,Downtown Toronto
32,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Restaurant,Bar,Beer Bar,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Smoothie Shop,Sandwich Place,M7A,Downtown Toronto
33,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Theater,Shoe Store,Brewery,Restaurant,M5A,Downtown Toronto
43,"The Annex, North Midtown, Yorkville",Coffee Shop,Sandwich Place,Café,Pub,Cheese Shop,Donut Shop,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,BBQ Joint,M5R,Central Toronto


It seems like the most expensive zone with several different restaurants, parks and gym

### Cluster 2: Normal Neighborhood

In [38]:
toronto_cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]-2))]]
toronto_cluster2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough
1,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Comfort Food Restaurant,Thai Restaurant,Boutique,Juice Bar,Restaurant,Butcher,Pub,M5M,North York
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Bakery,Cheese Shop,Restaurant,Eastern European Restaurant,Beach,M5E,Downtown Toronto
3,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,M6K,West Toronto
5,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,M5V,Downtown Toronto
9,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Pub,Smoke Shop,Café,Yoga Studio,M4Y,Downtown Toronto
11,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Japanese Restaurant,Deli / Bodega,Seafood Restaurant,Italian Restaurant,M5L,Downtown Toronto
14,Don Mills,Gym,Beer Store,Coffee Shop,Restaurant,Japanese Restaurant,Caribbean Restaurant,Sandwich Place,Bike Shop,Sporting Goods Shop,Italian Restaurant,M3B,North York
14,Don Mills,Gym,Beer Store,Coffee Shop,Restaurant,Japanese Restaurant,Caribbean Restaurant,Sandwich Place,Bike Shop,Sporting Goods Shop,Italian Restaurant,M3C,North York
17,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Shoe Store,Restaurant,Japanese Restaurant,Food Court,Chinese Restaurant,Juice Bar,Bank,M2J,North York
18,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,Asian Restaurant,Deli / Bodega,Salad Place,Seafood Restaurant,M5X,Downtown Toronto


Classic neighborhood with a mix of bar, pubs, restaurants and shops

### Cluster 3: Airport

In [39]:
toronto_cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]-2))]]
toronto_cluster3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough
8,Christie,Grocery Store,Café,Park,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub,Coffee Shop,M6G,Downtown Toronto
15,Downsview,Grocery Store,Park,Airport,Shopping Mall,Baseball Field,Coffee Shop,Business Service,Discount Store,Home Service,Bank,M3K,North York
15,Downsview,Grocery Store,Park,Airport,Shopping Mall,Baseball Field,Coffee Shop,Business Service,Discount Store,Home Service,Bank,M3L,North York
15,Downsview,Grocery Store,Park,Airport,Shopping Mall,Baseball Field,Coffee Shop,Business Service,Discount Store,Home Service,Bank,M3M,North York
15,Downsview,Grocery Store,Park,Airport,Shopping Mall,Baseball Field,Coffee Shop,Business Service,Discount Store,Home Service,Bank,M3N,North York


This Cluster includes all the airport and surrondings